In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import altair as alt

In [ ]:

# Load polygons and points
seattle_cras = gpd.read_file("cras.geojson")
eca_liquefaction = gpd.read_file("polygons.geojson")
eca_slide = gpd.read_file("polygons_set2.geojson")
retrofits = gpd.read_file("points.geojson")  # or use pd.read_csv("points.csv")
urms = gpd.read_file("points2.geojson")

# If points are from CSV, convert to GeoDataFrame
# points = pd.read_csv("points.csv")
# points = gpd.GeoDataFrame(
#     points,
#     geometry=gpd.points_from_xy(points.longitude, points.latitude),
#     crs="EPSG:4326"
# )

# (A) Assign labels to polygons
polygons["label"] = [
    f"Polygon_{i}" for i in range(len(polygons))
]

# (B) Spatial join: which polygon each point falls into
points_in_polygons = gpd.sjoin(
    points,
    polygons,
    how="left",
    predicate="within"
)

counts_per_polygon = (
    points_in_polygons.groupby("label")
    .size()
    .reset_index(name="count")
)

print(counts_per_polygon)

# (C) Total points inside polygons
total_in_polygons = counts_per_polygon["count"].sum()
print("Total points inside polygons:", total_in_polygons)

# (D) Points outside polygons
outside_count = points_in_polygons["label"].isna().sum()
print("Points outside polygons:", outside_count)

In [ ]:
# Load two sets of polygons
polygons1 = gpd.read_file("polygons_set1.geojson")
polygons2 = gpd.read_file("polygons_set2.geojson")

# Ensure both are in the same CRS
polygons1 = polygons1.to_crs("EPSG:3857")
polygons2 = polygons2.to_crs("EPSG:3857")

# Compute intersections
overlaps = gpd.overlay(
    polygons2,
    polygons1,
    how="intersection"
)

# Calculate area of overlaps
overlaps["overlap_area"] = overlaps.geometry.area

# Sum overlap area for each polygon in set2
overlap_summary = (
    overlaps.groupby(polygons2.index.name or "index_left")["overlap_area"]
    .sum()
    .reset_index()
)

print(overlap_summary)